# 2020-01-28

In this notebook I will test the pickling process (open a file), then move on to trying to do sentiment analysis on the comments:

In [1]:
import pickle
with open('comms_spa.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [1]:
import pickle
import pandas as pd
import random
import numpy as np
import pandas as pd
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm.notebook import tqdm
from pprint import pprint

C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\thinc\neural\train.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from .optimizers import Adam, SGD, linear_decay
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\thinc\check.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import defaultdict, Sequence, Sized, Iterable, Callable
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\thinc\check.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import defaultdict, Sequence, Sized, Iterable, Callable


In [23]:
mynewlist[10]

i love every single human in this video!!

In [20]:
type(mynewlist[10])

spacy.tokens.doc.Doc

### So the pickling process works, now onto linking the comments found with the commenter / video / channel details

In [2]:
insight_dir = 'C:\\Users\\Ronald Maj\\Documents\\GitHub\\InsightDataProject\\'

# Load vids data
vids_df = pd.read_csv(insight_dir+'data\\processed\\all_vids_en_df.csv', index_col=0)
# Load comments data
comms_df = pd.read_csv(insight_dir+'data\\processed\\all_comments_en_df.csv', index_col=0)

C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
comms_df = comms_df.dropna(subset=['textDisplay'])

In [4]:
comms_df = comms_df.drop(['Unnamed: 0.1'], axis=1)

In [5]:
# For our new sample, just get a random assortment of 10,000 comments from teh data frame:
docs_samp_df = comms_df.sample(10000)

In [6]:
docs_samp_df.head(5)

CommID  \
912330                         UgwB74R-ao2HxuUw9FV4AaABAg   
584253  UgyaEeeU7m4_jqndmoN4AaABAg.8zK7iRmI5S88zOPF69T1YO   
634121                         Ugyk96C9tOXQVWQdeqd4AaABAg   
279707  UgwcfnNqmyZIq5pIVbR4AaABAg.8s-PnIYPKiC8yztHlWC_YI   
109042                         UgwzVCS-rexbv7FGDvt4AaABAg   

                              authorChannelId  \
912330  {'value': 'UCCjezOkAnUm7d72_6RSQjWw'}   
584253  {'value': 'UCByC7nfOlb-qpy1DpL38zBA'}   
634121  {'value': 'UCmvH1SQnoGvL0n7qFn91uPw'}   
279707  {'value': 'UCdu7UGB7YG8tBuQ0cHCm5xA'}   
109042  {'value': 'UCva8ubjNzA2qXNHvA4GLACQ'}   

                                         authorChannelUrl authorDisplayName  \
912330  http://www.youtube.com/channel/UCCjezOkAnUm7d7...     Fortnut playz   
584253  http://www.youtube.com/channel/UCByC7nfOlb-qpy...      djangoDakoti   
634121  http://www.youtube.com/channel/UCmvH1SQnoGvL0n...          Amanda P   
279707  http://www.youtube.com/channel/UCdu7UGB7YG8tBu...    Denis Breimann   
109042  http://www.youtube.com/channel/UCva8ubjNzA2qXN...   Christine Khamo   

                                    authorProfileImageUrl  canRate  likeCount  \
912330  https://yt3.ggpht.com/a/AGF-l7-egKOq6WZ9wsH6wv...     True          0   
584253  https://yt3.ggpht.com/a/AGF-l7_H3YD8h50Gdz3aOE...     True          1   
634121  https://yt3.ggpht.com/a/AGF-l78qEU9IqYRTa3fPxh...     True        418   
279707  https://yt3.ggpht.com/a/AGF-l7_crRb50K8tASA7ny...     True          0   
109042  https://yt3.ggpht.com/a/AGF-l7_JCOKg9HgPgSJ6AQ...     True          3   

       moderationStatus                    parentId               publishedAt  \
912330              NaN                           0  2019-09-13T19:55:35.000Z   
584253              NaN  UgyaEeeU7m4_jqndmoN4AaABAg  2019-09-02T19:17:27.000Z   
634121              NaN                           0  2019-10-30T01:21:01.000Z   
279707              NaN  UgwcfnNqmyZIq5pIVbR4AaABAg  2019-08-23T21:28:26.000Z   
109042              NaN                           0  2019-09-14T12:04:01.000Z   

                                              textDisplay  \
912330                       0:50 is that voice of zelda?   
584253                                      @MEGACHEESE 😘   
634121  Pink and Red seem like good choices.\n\nThe do...   
279707                That's not comparable, like, at all   
109042  Always love to chill and watch your videos. Su...   

                                             textOriginal  \
912330                       0:50 is that voice of zelda?   
584253                                      @MEGACHEESE 😘   
634121  Pink and Red seem like good choices.\n\nThe do...   
279707                That's not comparable, like, at all   
109042  Always love to chill and watch your videos. Su...   

                       updatedAt      videoId viewerRating  
912330  2019-09-13T19:55:35.000Z  YSqHZUCkFfQ         none  
584253  2019-09-02T19:17:27.000Z  od7P-RhLjLQ         none  
634121  2019-10-30T01:21:17.000Z  iglH3Q2IGj8         none  
279707  2019-08-23T21:28:26.000Z  817R3ApVFok         none  
109042  2019-09-14T12:04:01.000Z  TVN9Jwh9Nwc         none

In [13]:
comm_docs = docs_samp_df['textDisplay']

In [14]:
def clean_text(text):
    # lower case:
    text = text.lower()
    
    # replace new line and return with space
    text = text.replace("\n", " ").replace("\r"," ")
    
    # replace punctuation marks with space
    punc_list = '#/<>@[\]{}|^~'
    t=str.maketrans(dict.fromkeys(punc_list, ""))
    
    return text

comm_docs = [clean_text(doc) for doc in comm_docs]


# Cleaning emojis
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

comm_docs = [deEmojify(doc) for doc in comm_docs]

<>:9: DeprecationWarning: invalid escape sequence \]
<>:9: DeprecationWarning: invalid escape sequence \]
<>:9: DeprecationWarning: invalid escape sequence \]
<ipython-input-14-25fd158cf537>:9: DeprecationWarning: invalid escape sequence \]
  punc_list = '#/<>@[\]{}|^~'


In [15]:
comm_docs[0:5]

['0:50 is that voice of zelda?',
 '@megacheese ',
 'pink and red seem like good choices.  the dogs like both of them, red is really chill and pink likes to say hi and give kisses (so sweet).  red is really unique. pink has good relationships and is also unique (and bi-eyed).',
 "that's not comparable, like, at all",
 'always love to chill and watch your videos. such a stress reliever! ']

### Load spaCy language model

In [16]:
nlp2 = spacy.load("en_core_web_lg")

In [18]:
list_attrs = ["ID", 
              "NORM", 
              "LOWER", 
              "SHAPE",
              "PREFIX", 
              "SUFFIX", 
              "CLUSTER", 
              "LENGTH", 
              "POS", 
              "LEMMA", 
              "TAG", 
              "DEP", 
              "HEAD", 
              "ENT_IOB", 
              "ENT_TYPE", 
              "ENT_ID", 
              "ENT_KB_ID", 
              "SENT_START"]

In [19]:
from spacy.tokens import DocBin
doc_bin = DocBin(attrs=list_attrs)

ImportError: cannot import name 'DocBin' from 'spacy.tokens' (C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\spacy\tokens\__init__.py)

In [21]:
len(comms_df)

935897

In [ ]:
# Empty matrix
comms_vec = np.zeros((len(comms_df),300))

In [22]:
len(comm_docs)

10000

In [23]:
spa_docs= []
for doc in tqdm(comm_docs):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp2(doc)
    spa_docs.append(pr)

In [192]:
eg_doc = nlp2('canon')

In [193]:
# Create a matrix of the search term/s doc:
eg_vec = eg_doc.vector.reshape(1,-1)

In [194]:
# Create a matrix of the database:

# Empty matrix
comms_vec = np.zeros((len(spa_docs),300))

# Go through each doc and save the word2vec vector to the matrix:
for idx,doc in enumerate(spa_docs):
    comms_vec[idx] = doc.vector.reshape(1,-1)

In [195]:
from sklearn.metrics.pairwise import cosine_similarity

In [196]:
sim_mat = cosine_similarity(eg_vec,comms_vec)

In [197]:
len(sim_mat[0])

10000

In [209]:
tf_ar = sim_mat > 0.35

In [210]:
result = [(sim_mat[0][idx],x) for idx,x in enumerate(spa_docs) if tf_ar[0][idx]]

In [211]:
def getKey(item):
    return item[0]

In [212]:
srt_result = sorted(result, key=getKey, reverse=True)

In [213]:
for sim,res in srt_result:
    print(sim)
    print(res)
    print('\n')

0.4205289471920324
i would love to see your review of the canon eosr!


0.3987104587629539
you say about beautifully peter, also yes do a review on the eos rp


0.37363115084355597
love from hp


0.36348023352497266
amazing :). the macro lens is super cool


0.36098596288225115
@baby yoda yt kaylee no.. not scanner, scummy.


0.3583458510817653
well now i wanna buy the mavic zoom...


0.356971343041984
harry potter crafts *hawaii* idk why yall getting hawaii wrong


0.3541878840378725
this movie is clearly pro mushroom, anti laptop propaganda




Connecting now with the channel it is from:

In [214]:
docs_samp_df[tf_ar[0]]

CommID  \
56716                          Ugwsgdqe1yH54Fa1T554AaABAg   
56741                          UgxkIWtrFI_Vf1OQC_J4AaABAg   
58228                          Ugz-NMnVjI-VBtTUYNV4AaABAg   
304938                         Ugz5zPbIdCeEfnvlan14AaABAg   
259825                         UgwQaxL8bbytoRHPvvh4AaABAg   
725017  Ugz715Bh1kVfEhJkahN4AaABAg.932rvxtw-hy933G6jNL680   
137273                         UgwXMJQnCnqmi6Xu1Xl4AaABAg   
538627  Ugwf46AQ_D9qgU4Pz-Z4AaABAg.948_xQocvHH948xVHCC29e   

                              authorChannelId  \
56716   {'value': 'UC26SB-Czj1sEtf8MEQhNDgw'}   
56741   {'value': 'UCt238tWsYad5PLMjumUl0oQ'}   
58228   {'value': 'UCNh_vWHfHBwblVPNDYaZcWQ'}   
304938  {'value': 'UCuJNw8K9ZeqP3tICWcN4WWA'}   
259825  {'value': 'UCS7brNNtNbqasPit5dI0EvQ'}   
725017  {'value': 'UCNMxVgLQZZn4H3X8DZwJrpA'}   
137273  {'value': 'UCedmGfBktNiFYa9w30DFH4g'}   
538627  {'value': 'UCpLojdmXIKurqQiVUr8bYSw'}   

                                         authorChannelUrl authorDisplayName  \
56716   http://www.youtube.com/channel/UC26SB-Czj1sEtf...    Curtis Distaso   
56741   http://www.youtube.com/channel/UCt238tWsYad5PL...    Ben Gathercole   
58228   http://www.youtube.com/channel/UCNh_vWHfHBwblV...       Omri Leshem   
304938  http://www.youtube.com/channel/UCuJNw8K9ZeqP3t...        Kath Flynn   
259825  http://www.youtube.com/channel/UCS7brNNtNbqasP...       Violet Bird   
725017  http://www.youtube.com/channel/UCNMxVgLQZZn4H3...     Alexa Cabello   
137273  http://www.youtube.com/channel/UCedmGfBktNiFYa...       ajay sharma   
538627  http://www.youtube.com/channel/UCpLojdmXIKurqQ...               ARE   

                                    authorProfileImageUrl  canRate  likeCount  \
56716   https://yt3.ggpht.com/a/AGF-l7_XtppEwRg-zCKzLl...     True          9   
56741   https://yt3.ggpht.com/a/AGF-l7-JdmJ7lFP_J0mhWq...     True          0   
58228   https://yt3.ggpht.com/a/AGF-l79KwnCchcQ8qg-nAw...     True          1   
304938  https://yt3.ggpht.com/a/AGF-l79YbeCyNFpJYqMdYo...     True       4450   
259825  https://yt3.ggpht.com/a/AGF-l78ylVH9sQEMIi20Tv...     True          0   
725017  https://yt3.ggpht.com/a/AGF-l7_1hckVeH0H-JmwaV...     True          1   
137273  https://yt3.ggpht.com/a/AGF-l78D56QuU7uJ_F3s9h...     True          0   
538627  https://yt3.ggpht.com/a/AGF-l7_FEJEClF26PgGd1h...     True          1   

       moderationStatus                    parentId               publishedAt  \
56716               NaN                           0  2019-12-02T23:25:20.000Z   
56741               NaN                           0  2019-12-04T09:02:47.000Z   
58228               NaN                           0  2019-10-30T18:54:02.000Z   
304938              NaN                           0  2019-06-01T01:53:47.000Z   
259825              NaN                           0  2020-01-23T22:34:31.000Z   
725017              NaN  Ugz715Bh1kVfEhJkahN4AaABAg  2019-12-27T20:50:35.000Z   
137273              NaN                           0  2020-01-07T14:32:06.000Z   
538627              NaN  Ugwf46AQ_D9qgU4Pz-Z4AaABAg  2020-01-23T22:26:00.000Z   

                                              textDisplay  \
56716   You say about beautifully Peter, also yes do a...   
56741   I would LOVE to see your review of the Canon E...   
58228              Well now i wanna buy the Mavic Zoom...   
304938  This movie is clearly pro mushroom, anti lapto...   
259825           Amazing :). The macro lens is super cool   
725017  Harry Potter Crafts *Hawaii* idk why yall gett...   
137273                                 Love from hp❤️❤️❤️   
538627     @Baby Yoda YT Kaylee no.. Not scanner, scummy.   

                                             textOriginal  \
56716   You say about beautifully Peter, also yes do a...   
56741   I would LOVE to see your review of the Canon E...   
58228              Well now i wanna buy the Mavic Zoom...   
304938  This movie is clearly pro mushroom, anti lapto...   
259825           Amazing :). The

In [215]:
sorted(docs_samp_df[tf_ar[0]]['videoId'])

['DiSuDhIxk88',
 'RgLP01RuL_M',
 '_B5OZzydoJU',
 'eKfpM4cBfwo',
 'eKfpM4cBfwo',
 'hCa_ddzU6w4',
 'luNn6ovoKVw',
 'q9VkIgFmxuk']

In [ ]:
vids_df

In [216]:
vids_df.loc[vids_df['VidID'].isin(sorted(docs_samp_df[tf_ar[0]]['videoId']))]

Unnamed: 0.1        VidID                 ChannelID  \
370            370  eKfpM4cBfwo  UC3DkFux8Iv-aYnTRWzwaiBA   
376            376  luNn6ovoKVw  UC3DkFux8Iv-aYnTRWzwaiBA   
935            935  RgLP01RuL_M  UC15sF31QE-7rooIlSKnTq_w   
3290          3290  DiSuDhIxk88  UCey_c7U86mJGz1VJWH5CYPA   
3320          3320  hCa_ddzU6w4  UCTSRIY3GLFYIpkR2QwyeklA   
3405          3405  q9VkIgFmxuk  UC7zsxKqd5MicTf4VhS9Y74g   
3989          3989  _B5OZzydoJU  UCrbSjN1_5U7hqFKRA0nvFXQ   

                                             VidTitle  \
370     WHAT TO BUY A PHOTOGRAPHER 2019 (or yourself)   
376   Mavic Mini - How small will these ACTUALLY get?   
935                             I AM NOT PAKISTANI 🇮🇳   
3290        Top iPhone Accessories For Making Videos!   
3320            Revisiting Jake Paul's Team 1000 Scam   
3405       Fateful Findings: An Extremely Weird Movie   
3989      COLLEEN BALLINGER'S CHRISTMAS SPECIAL 2019!   

                                         VidDescription  \
370   BUCKET SHOT COFFEE BOX : https://bit.ly/2RaDjv...   
376   Mavic Mini : https://click.dji.com/AAoEWQel7JG...   
935   This vlog is about 1 Million Celebration and m...   
3290  Showing you guys some of my favorite iPhone ac...   
3320  Go to https://expressvpn.com/drew, to take bac...   
3405  Go to https://zebit.com/kurtistown for up to $...   
3989  Come see me on tour! - http://mirandasings.com...   

                  VidPublished  viewCount  likeCount  dislikeCount  \
370   2019-12-02T21:43:57.000Z   530562.0    22519.0         623.0   
376   2019-10-30T13:01:11.000Z   496868.0    18915.0         281.0   
935   2019-12-29T15:24:01.000Z   388111.0    40586.0         787.0   
3290  2020-01-23T18:14:47.000Z   158586.0     7483.0         142.0   
3320  2020-01-23T17:47:56.000Z  1633792.0   121294.0         711.0   
3405  2019-05-31T19:00:11.000Z  1845485.0   114820.0         467.0   
3989  2019-12-27T17:00:06.000Z   723713.0    31676.0         512.0   

      favoriteCount  commentCount  
370             0.0        2679.0  
376             0.0        1592.0  
935             0.0        3202.0  
3290            0.0         465.0  
3320            0.0        5401.0  
3405            0.0        9702.0  
3989            0.0        2393.0

In [218]:
vids_df.loc[vids_df['VidID'].isin(sorted(docs_samp_df[tf_ar[0]]['videoId']))]['ChannelID']

370     UC3DkFux8Iv-aYnTRWzwaiBA
376     UC3DkFux8Iv-aYnTRWzwaiBA
935     UC15sF31QE-7rooIlSKnTq_w
3290    UCey_c7U86mJGz1VJWH5CYPA
3320    UCTSRIY3GLFYIpkR2QwyeklA
3405    UC7zsxKqd5MicTf4VhS9Y74g
3989    UCrbSjN1_5U7hqFKRA0nvFXQ
Name: ChannelID, dtype: object

Just realised I don't have the channels information, but it would follow the pattern as above

Need to start processing the word embeddings for tomorrow

In [220]:
all_comm_docs = comms_df['textDisplay']

In [221]:
len(all_comm_docs)

935897

In [223]:
for i in range(0,95):
    spa_docs= []
    for doc in tqdm(all_comm_docs[i:i+10000]):
        # Passes that article through the pipeline and adds to a new list.
        pr = nlp2(doc)
        spa_docs.append(pr)
    with open('D:\\Pickle_files-Embedded-YT-Comments\\comms_spa_'+str(i+1)+'.pkl', 'wb') as f:
        pickle.dump(spa_docs, f)

### Above forgot to multiply the index value by 10,000 so didn't actually create what I thought I did